## Make necessary imports

In [1]:
import sys
sys.path.insert(0, '../')

In [2]:
import time
import numpy as np
np.set_printoptions(precision=3, linewidth=200, suppress=True)
import tensorflow as tf

In [3]:
from library.datasets.cifar10 import CIFAR10
from library.plot_tools import plot
from library.utils import file_utils
from library.tf.mlp_classifier import MLPClassifier

## Select Tensorflow device

In [4]:
# from tensorflow.python.client import device_lib
# local_device_protos = device_lib.list_local_devices()
# cpu_devices = [x.name for x in local_device_protos if x.device_type == 'CPU']
# gpu_devices = [x.name for x in local_device_protos if x.device_type == 'GPU']
# print('Available CPU Devices: ', end='')
# print(cpu_devices)
# print('Available GPU Devices: ', end='')
# print(gpu_devices)
# if len(gpu_devices) == 0:
#     if len(cpu_devices) > 0:
#         device_name = '/cpu:0'
#         print('Using CPU: %s' %device_name)
#     else:
#         print('No CPU present in the system!!!')
# else:
#     device_name = '/gpu:0'
#     print('Using GPU: %s' %device_name)

In [5]:
total_time = 0

## Experiment

In [6]:
exp_no = 10
data_source = 'Website'
num_images_required = 1.0
file_no = 103

### Parameters for experiment

In [7]:
device_name = '/gpu:0'
learn_rate = 1.5
train_epochs = 100
display_step = 1
reg_const = 0.01
train_val_split_data = None
train_val_split = 0.8
transform = True
transform_method = 'StandardScaler'
learn_rate_type = 'constant'
dataset = 'cifar10'
descent_method = 'gradient'

### Log directories

In [8]:
log_dir = '../logs/' + dataset + '/' + str(file_no).zfill(3) + '_tf_mlp_raw/exp_no_' + str(exp_no).zfill(3) + '/'
log_file = log_dir + 'mlp_classifier.ckpt'
model_file = log_dir + 'mlp_classifier.pb'
print('Writing tensorboard logs to %s' % log_file)
print('View logs by running tensorboard: ', end='')
print('\"tensorboard --logdir=\'./%s/103_tf_mlp_raw/\' --port 61111\"' %dataset)

Writing tensorboard logs to ../logs/cifar10/103_tf_mlp_raw/exp_no_010/mlp_classifier.ckpt
View logs by running tensorboard: "tensorboard --logdir='./cifar10/103_tf_mlp_raw/' --port 61111"


## Step 1: Load CIFAR 10 Dataset

In [9]:
start = time.time()
one_hot = True
cifar10 = CIFAR10(one_hot_encode=one_hot, num_images=num_images_required, preprocess='StandardScaler',
                  train_validate_split=train_val_split_data, endian='little')
cifar10.load_data(train=True, test=True, data_directory='./datasets/cifar10/')
end = time.time()
print('[ Step 1] Loaded CIFAR 10 Dataset in %.4f ms' %((end-start)*1000))
total_time += (end-start)

Loading CIFAR 10 Dataset
MD5sum of the file: ./datasets/cifar10/cifar-10.tar.gz is verified
Loading 50000 train images
Loading CIFAR 10 Training Dataset
Reading unpicked data file: ./datasets/cifar10/cifar-10-batches/data_batch_1
Reading unpicked data file: ./datasets/cifar10/cifar-10-batches/data_batch_2
Reading unpicked data file: ./datasets/cifar10/cifar-10-batches/data_batch_3
Reading unpicked data file: ./datasets/cifar10/cifar-10-batches/data_batch_4
Reading unpicked data file: ./datasets/cifar10/cifar-10-batches/data_batch_5


/net/voxel03/misc/me/praneethas/Softwares/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


Loading 10000 test images
Loading CIFAR 10 Test Dataset
Unpickling test file: ./datasets/cifar10/cifar-10-batches/test_batch
Reading unpicked test file: ./datasets/cifar10/cifar-10-batches/test_batch
Loaded CIFAR 10 Dataset in 8.5198 seconds
[ Step 1] Loaded CIFAR 10 Dataset in 8521.2324 ms


In [10]:
# print('Train data shape:', cifar10.train.data.shape)
# if one_hot is True:
#     print('Train labels shape:', cifar10.train.one_hot_labels.shape)
# print('Train class labels shape:', cifar10.train.class_labels.shape)
# if train_val_split_data is not None:
#     print('Validate data shape:', cifar10.validate.data.shape)
#     if one_hot is True:
#         print('Validate labels shape:', cifar10.validate.one_hot_labels.shape)
#     print('Validate class labels shape:', cifar10.validate.class_labels.shape)
# print('Test data shape:', cifar10.test.data.shape)
# if one_hot is True:
#     print('Test labels shape:', cifar10.test.one_hot_labels.shape)
# print('Test class labels shape:', cifar10.test.class_labels.shape)

In [11]:
# print('Training images')
# print(cifar10.train.data[:5])
# if one_hot is True:
#     print('Training labels')
#     print(cifar10.train.one_hot_labels[:5])
# print('Training classes')
# print(cifar10.train.class_labels[:5])
# print('Testing images')
# print(cifar10.test.data[:5])
# if one_hot is True:
#     print('Testing labels')
#     print(cifar10.test.one_hot_labels[:5])
# print('Testing classes')
# print(cifar10.test.class_labels[:5])

## Step 1.1 Load sample images

In [12]:
# cifar10.plot_sample(plot_data=True, plot_test=True, fig_size=(7, 7))

In [13]:
# cifar10.plot_images(cifar10.train.data[:50, :], cifar10.train.class_names[:50], 
#                     nrows=5, ncols=10, fig_size=(20,50), fontsize=35, convert=True)

## Step 2: Linear Regression

### Step 2.1: Model linear regression y = Wx + b

In [14]:
num_features = cifar10.train.data.shape[1]
num_classes = 10

In [15]:
network_config = {'layer_1': {'weight': {'name': 'Weight_1', 'type': 'random_normal'},
                              'bias': {'name': 'Bias_1', 'type': 'random_normal'},
                              'activation_fn': 'sigmoid',
                              'num_nodes': 3072,
                              'layer_name': 'Layer_1'
                              },
                  'layer_2': {'weight': {'name': 'Weight_2', 'type': 'random_normal'},
                              'bias': {'name': 'Bias_2', 'type': 'random_normal'},
                              'activation_fn': 'sigmoid',
                              'num_nodes': 3072,
                              'layer_name': 'Layer_2'
                              },
                  'output_layer': {'weight': {'name': 'Output_Weight', 'type': 'random_normal'},
                                   'bias': {'name': 'Output_Bias', 'type': 'random_normal'},
                                   'activation_fn': 'sigmoid',
                                   'layer_name': 'Output_Layer'
                                  }
                 }

In [16]:
mlp = MLPClassifier(verbose=False, device=device_name, session_type='interactive', num_iter=train_epochs, 
                    err_tolerance=1e-7, train_validate_split=train_val_split, display_step=display_step,
                    learn_step=10, learn_rate_type=learn_rate_type, learn_rate=learn_rate, reg_const=reg_const, 
                    logs=True, log_dir=log_dir, test_log=True, save_model=True, checkpoint_filename=log_file, 
                    save_checkpoint=True, restore=False, model_name=model_file, config=network_config)

### Step 2.2: Create the tensorflow graph

In [17]:
start = time.time()
mlp.create_graph(num_features=num_features, num_classes=num_classes)
end = time.time()
print('Generated the tensorflow graph in %.4f ms' %((end-start)*1000))
total_time = (end-start)

Session: <tensorflow.python.client.session.InteractiveSession object at 0x7f511aa04748>
Model has been trained for 0 iterations
Tensorflow graph created in 1.0844 seconds
Generated the tensorflow graph in 1084.7902 ms


In [18]:
mlp.print_parameters()

Parameters for MLP classifier
>> Input Parameters
Input                  : Tensor("Inputs/Data/X_input:0", shape=(?, 3072), dtype=float32, device=/device:GPU:0) 
True one hot labels    : Tensor("Inputs/Train_Labels/y_true_one_hot_label:0", shape=(?, 10), dtype=float32, device=/device:GPU:0) 
True class             : Tensor("Inputs/Train_Labels/y_true_class:0", shape=(?,), dtype=int64, device=/device:GPU:0) 
Predict one hot labels : Tensor("Predictions/one_hot:0", shape=(?, 10), dtype=float32, device=/device:GPU:0) 
Predict class          : Tensor("Predictions/ArgMax:0", shape=(?,), dtype=int64, device=/device:GPU:0) 
>> Layer 1
	Weights : Tensor("Parameters/Weights/Weight_1/read:0", shape=(3072, 3072), dtype=float32, device=/device:GPU:0)
	Bias    : Tensor("Parameters/Bias/Bias_1/read:0", shape=(3072,), dtype=float32, device=/device:GPU:0)
	Layer   : Tensor("Hidden_Layers/Layer_1_1:0", shape=(?, 3072), dtype=float32, device=/device:GPU:0)
>> Layer 2
	Weights : Tensor("Parameters/Weight

### Step 2.3: Fit the model/training

In [19]:
start = time.time()
mlp.fit(cifar10.train.data, cifar10.train.one_hot_labels, cifar10.train.class_labels,
        test_data=cifar10.test.data, test_labels=cifar10.test.one_hot_labels, test_classes=cifar10.test.class_labels)
end = time.time()
print('Fit completed in %.4f seconds' % (end-start))

Length of train loss          : 0
Length of train accuracy      : 0
Length of validate loss       : 0
Length of validate accuracy   : 0
Length of test accuracy       : 0
Restoring training from epoch : 0
>>> Epoch [  0/100]
train_loss: 2.1459 | train_acc: 0.2031 | val_loss: 2.2145 | val_acc: 0.1587 | test_acc: 0.1625 | Time: 6.5826 s
>>> Epoch [  1/100]
train_loss: 2.0881 | train_acc: 0.2188 | val_loss: 2.1499 | val_acc: 0.2132 | test_acc: 0.2152 | Time: 6.2144 s
>>> Epoch [  2/100]
train_loss: 2.0426 | train_acc: 0.3047 | val_loss: 2.1230 | val_acc: 0.2366 | test_acc: 0.2311 | Time: 6.2421 s
>>> Epoch [  3/100]
train_loss: 2.0148 | train_acc: 0.2734 | val_loss: 2.1092 | val_acc: 0.2456 | test_acc: 0.2451 | Time: 6.4061 s
>>> Epoch [  4/100]
train_loss: 2.0001 | train_acc: 0.3516 | val_loss: 2.0887 | val_acc: 0.2606 | test_acc: 0.2625 | Time: 6.2348 s
>>> Epoch [  5/100]
train_loss: 2.0066 | train_acc: 0.3516 | val_loss: 2.0786 | val_acc: 0.2683 | test_acc: 0.2718 | Time: 6.7619 s
>>> 

AssertionError: Predictions/predict_class is not in graph

### Step 2.4: Make the predictions

In [ ]:
prediction_numbers = mlp.predict(cifar10.test.data)
prediction_classes = []
num_test_images = cifar10.test.data.shape[0]
for i in range(num_test_images):
    prediction_classes.append(cifar10.classes[int(prediction_numbers[i])])

In [ ]:
cifar10.plot_images(cifar10.test.data[:50], cifar10.test.class_names[:50], cls_pred=prediction_classes[:50], 
                    nrows=5, ncols=10, fig_size=(20,50), fontsize=30, convert=True)

### Step 2.5: Print the results

In [ ]:
test_accuracy = mlp.score(cifar10.test.data, cifar10.test.class_labels)
print('Accuracy of the linear classifier on test dataset: %.4f' %test_accuracy)

In [ ]:
mlp.print_classification_results(cifar10.test.data, cifar10.test.one_hot_labels, cifar10.test.class_labels,
                                 test_class_names=cifar10.classes, normalize=True)

### Step 2.6: Plot results

In [ ]:
mlp.plot_loss()

In [ ]:
mlp.plot_accuracy()

### Step 2.7: Close the session

In [ ]:
mlp.close()

## Step 3: Write to file

In [ ]:
def output_HTML(read_file, output_file):
    from nbconvert import HTMLExporter
    import codecs
    import nbformat
    exporter = HTMLExporter()
    output_notebook = nbformat.read(read_file, as_version=4)
    print()
    output, resources = exporter.from_notebook_node(output_notebook)
    codecs.open(output_file, 'w', encoding='utf-8').write(output)

In [ ]:
%%javascript
var notebook = IPython.notebook
notebook.save_notebook()

In [ ]:
%%javascript
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "theNotebook = " + "'"+thename+"'";
kernel.execute(command);

In [ ]:
current_file = './' + theNotebook + '.ipynb'
output_file = log_dir + str(file_no).zfill(2) + '_exp_no_' + str(exp_no).zfill(3) + '_' + theNotebook + '.html'
print('Current file: ' + str(current_file))
print('Output file: ' + str(output_file))
file_utils.mkdir_p(log_dir) 
output_HTML(current_file, output_file)